<a href="https://colab.research.google.com/github/FedorTaggenbrock/data_intensive_systems/blob/main/notebooks/main_tests_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Handle importing/installing, both local and on Colab**

In [4]:
import sys
global ON_COLAB
ON_COLAB = 'google.colab' in sys.modules
if ON_COLAB:
    # Do stuff that only needs to happen on colab
    !pip install pyspark  # noqa
    !pip install ijson
    pass
else:
    # Do stuff that only needs to happen on local computer
    pass

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Rerun the code cell below to use the latest version of the python files!

In [7]:
import importlib

def import_modules():
    if ON_COLAB:
        print("Running in Colab: importing from GitHub.")
        # Clone the repository
        !git clone https://github.com/FedorTaggenbrock/data_intensive_systems.git

        # Change the current working directory to the cloned repository directory
        %cd /content/data_intensive_systems

        # Pull the latest changes from the repository
        !git pull origin main

        # Change back to the original working directory
        %cd /content

        # Add the path of the modules to sys.path
        sys.path.insert(0, "/content/data_intensive_systems/src")
    else:
        print("Running locally, importing from local.")
        # Append parent folder to path
        sys.path.append('../src')

    # Import and reload modules iteratively
    module_names = ['clustering',
                    'data_visualization',
                    'distance_functions',
                    'evaluate_clustering',
                    'generate_data',
                    'main_tests',
                    'non_spark_function_debugger',
                    # 'parse_data',
                    # 'parse_data_2',
                    'parse_data_3',
                    ]
    for module_name in module_names:
        print(module_name)

        module = importlib.import_module(module_name)
        importlib.reload(module)
        globals()[module_name] = module

import_modules()


Running in Colab: importing from GitHub.
fatal: destination path 'data_intensive_systems' already exists and is not an empty directory.
/content/data_intensive_systems
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0
Unpacking objects: 100% (4/4), 547 bytes | 547.00 KiB/s, done.
From https://github.com/FedorTaggenbrock/data_intensive_systems
 * branch            main       -> FETCH_HEAD
   e5f0129..8863601  main       -> origin/main
Updating e5f0129..8863601
Fast-forward
 src/non_spark_function_debugger.py | 15 ++++++++++-----
 1 file changed, 10 insertions(+), 5 deletions(-)
/content
clustering
data_visualization
distance_functions
evaluate_clustering
generate_data
main_tests
non_spark_function_debugger
parse_data_3


In [8]:
main_tests.run_all_tests()

Loading data


FileNotFoundError: ignored

Test all functions you want inside the run_all_tests() during development, for small sample sizes.


Code below is for actual result generation later, so that we can easily reuse intermediate values.

**Create a spark instance**

In [ ]:
spark_settings = {}
spark_instance = make_spark()

NameError: ignored

**Load and parse data**

In [ ]:
# Load and parse data
data_for_spark = parse_data()

**Perform clustering**

In [ ]:
# Run clustering and parameter tuning
clustering_settings = {
    'clustering_algorithm':'kmodes',
}
clustering_centroid_outcomes = run_clustering(
    spark_instance=spark_instance,
    data_for_spark, clustering_settings)

In [ ]:
# Evaluate clustering results
clustering_evaluation = evaluate_clustering(clustering_centroid_outcomes)

**Display results**

In [ ]:
# Display results (maybe only best result)
clustering_result = display_results(clustering_evaluation